In [1]:
import os, sys, random, datetime, json

from pyts.image import GramianAngularField

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
rootDir = "/Users/phillip/Documents/PhD/PhD3/Data/PhysioNet/dataset"
df_list = []

files = os.listdir(rootDir)
for file in files:
    df_list.append(pd.read_csv(os.path.join(rootDir, file), sep="|", header=0))

In [3]:
# 计算所有数据的平均值填充
# cell001_1
s = np.zeros(40).tolist()
c = np.zeros(40).tolist()

for e in df_list:
    s = s + e.sum()[:-1]
    c = c + e.count()[:-1]

a = s.div(c)

# 当不能用线性填充时,填充最近的,再不行用平均值填充
# cell001_2
threshold = 0  # the healthy-sepsis ratio threshold
data = []
i = 0
for e in df_list:
    # t = e.iloc[:, -1][e.iloc[:, -1] == 1].sum()
    # l = len(e)
    # if t / l > threshold:
    e.interpolate(inplace=True, limit_direction="both")
    e.interpolate(inplace=True, method="nearest")
    na_cols = e.columns[e.isna().any()]
    for col in na_cols:
        e.loc[e[col].isnull(), col] = a[col]
    data.append(e)

In [9]:
IMAGESIZE = 32
WINDOWSIZE = IMAGESIZE

In [10]:
infectedData_arr = []
normalData_arr = []

for i in range(len(data)):
    for j in range(0, len(data[i]) - WINDOWSIZE, 30):
        block = data[i].iloc[j : j + WINDOWSIZE, :-1].values
        if (data[i].SepsisLabel > 0).any():
            infectedData_arr.append(block)
        else:
            normalData_arr.append(block)

np.save(
    "normal_gramian_physionet_{}_{}.npy".format(IMAGESIZE, 40), np.array(normalData_arr)
)
np.save(
    "sepsis_gramian_physionet_{}_{}.npy".format(IMAGESIZE, 40),
    np.array(infectedData_arr),
)